# | default_exp core

In [ ]:
# | hide
# from bertopic import BERTopic
# from bertopic.vectorizers import OnlineCountVectorizer
import dagshub
from datetime import datetime
import dill as pickle
import dvc.api
# from hdbscan import HDBSCAN
from itertools import tee, islice, product
import joblib
# import mlflow
# from mlflow.models import infer_signature
import nbdev
from nbdev.showdoc import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import (
    CountVectorizer
    , TfidfTransformer
    , TfidfVectorizer
    , 
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from src.custom_sklearn_text_transformer_mlflow import CustomSKLearnAnalyzer
# from src.custom_stanza_mlflow import CustomSKLearnWrapper
import src.dataframe_preprocessor as dfpp
import stanza
from tqdm import tqdm
# from umap import UMAP

In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128'

# | export

In [ ]:
# | hide
# nbdev.nbdev_export()

### Data Preparation

In [ ]:
# instantiate stanza pipeline
stanza.download('en')
nlp = stanza.Pipeline('en', 
                    depparse_batch_size=50, 
                    depparse_min_length_to_batch_separately=50,
                    verbose=True,
                    use_gpu=True, # set to true when on cloud/not on streaming computer
                    batch_size=100
                    )

# load raw data and preprocess/clean
data = dvc.api.read(
    path='../data/recipes-en-201706/epicurious-recipes_m2.json'
    , mode='r')
raw_df = pd.read_json(data)
print('\n')
print('--------------')
print('Raw Dataframe:', end='\n')
print(raw_df.head())
print(raw_df.shape)


In [ ]:
# take sample and train/test split 
subset_df = raw_df.sample(n=100, random_state=45)
train_df, test_df = train_test_split(subset_df,test_size=0.5, random_state=45)

# pre_proc_df is cleaned dataframe
to_nlp_df = dfpp.preprocess_dataframe(train_df)
print('\n')
print('--------------')
print('Preprocessed Dataframe:', end='\n')
print(to_nlp_df.head())
print(to_nlp_df.shape)

In [ ]:
# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_transformer_params = {
    'analyzer': 'word',
    'ngram_range': (1,4),
    'min_df':3,
    'binary':False
}

sklearn_transformer = TfidfVectorizer(**sklearn_transformer_params)

# print('\n')
# print('-' * 80)
# print('sklearn fit transform on ingredients:', end='\n')

model_input = to_nlp_df['ingredients'].apply(" ".join).str.lower()

print("sklearn fit transform start: " + str(datetime.now()))

# Do fit transform on data
response = sklearn_transformer.fit_transform(tqdm(model_input)) 

print("sklearn fit transform end: " + str(datetime.now()))

transformed_recipe = pd.DataFrame(
        response.toarray(),
        columns=sklearn_transformer.get_feature_names_out(),
        index=model_input.index
)

print(transformed_recipe)
print(transformed_recipe.columns)

In [ ]:
# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_transformer_params = {    
    'analyzer': CustomSKLearnAnalyzer.ngrams_maker(
        min_ngram_length=1,
        max_ngram_length=4
        ),
    'min_df':3,
    'binary':False
}

sklearn_transformer = TfidfVectorizer(**sklearn_transformer_params)

model_input = to_nlp_df['ingredients_lemmafied']

# Do fit transform on data
print("fit_transform start: " + str(datetime.now()))
response = sklearn_transformer.fit_transform(tqdm(model_input)) 
print("fit_transform end: " + str(datetime.now()))

transformed_recipe = pd.DataFrame(
        response.toarray(),
        columns=sklearn_transformer.get_feature_names_out(),
        index=model_input.index
)

print(transformed_recipe.columns)

In [ ]:
transformed_recipe

In [ ]:
# load from MLflow
mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=f'https://dagshub.com/{DAGSHUB_USER_NAME}/MeaLeon.mlflow')

# cv_params are parameters for the sklearn CountVectorizer or TFIDFVectorizer
sklearn_transformer_params = {
    'analyzer': CustomSKLearnAnalyzer.ngrams_maker(
        min_ngram_length=1,
        max_ngram_length=4
        ),
    'min_df':3,
    'binary':True
}

# pipeline_params are parameters that will be logged in MLFlow and are a superset of library parameters
pipeline_params = {
    'stanza_model': 'en',
    'sklearn-transformer': 'Tf'
}

# update the pipeline parameters with the library-specific ones so that they show up in MLflow Tracking
pipeline_params.update(sklearn_transformer_params)
# pipeline_params.update(bertopic_params)

# signature = infer_signature(model_input=to_nlp_df['ingredients'],
#                             )

with mlflow.start_run(experiment_id=mlflow_exp_id):    
    # LOG PARAMETERS
    mlflow.log_params(pipeline_params)

    # LOG INPUTS (QUERIES) AND OUTPUTS
    # MLflow example uses a list of strings or a list of str->str dicts
    # Will be useful in STAGING/Evaluation
    
    # LOG MODEL
    # Instantiate sklearn OneHotEncoder
    sklearn_transformer = CountVectorizer(**sklearn_transformer_params)

    print('\n')
    print('-' * 80)
    print('sklearn fit transform on ingredients:', end='\n')

    model_input = to_nlp_df['ingredients']

    print('\n')
    print('-' * 80)
    print('Input Data: ', end='\n')
    print(model_input)

    print('\n')
    print('-' * 80)
    print('Input Data Shape: ', end='\n')
    print(model_input.shape)

    print('\n')
    print('-' * 80)
    print('Random 3 Records from Input Data: ', end='\n')
    print(model_input.sample(3, random_state=200))

    # Do fit transform on data
    response = sklearn_transformer.fit_transform(tqdm(model_input)) 
    
    transformed_recipe = pd.DataFrame(
            response.toarray(),
            columns=sklearn_transformer.get_feature_names_out(),
            index=model_input.index
    )

    signature = infer_signature(model_input=model_input,
                                model_output=transformed_recipe
                                )

    print('\n')
    print('-' * 80)
    print('Transformed Data:', end='\n')
    print(transformed_recipe)
    
    # mlflow.pyfunc.save_model(
    #     path=model_directory,
    #     code_path=["../src/"],
    #     python_model=CustomSKLearnWrapper(),
    #     input_example=to_nlp_df['ingredients'][0],    
    #     artifacts=artifacts
    # )

     # joblib.dump(sklearn_transformer, sklearn_transformer_path)
    with open(sklearn_transformer_path, "wb") as fo:
        pickle.dump(sklearn_transformer, fo)
        # mlflow.log_artifact(sklearn_transformer_path,
        #                     artifact_path='sklearn_transformer')

    # joblib.dump(transformed_recipe, transformed_recipes_path)
    with open(transformed_recipes_path, "wb") as fo:
        pickle.dump(transformed_recipe, fo)
        # mlflow.log_artifact(transformed_recipes_path,
        #                     artifact_path='transformed_recipes')


    model_info = mlflow.pyfunc.log_model( 
        code_path=["../src/"],
        python_model=CustomSKLearnWrapper(),
        input_example=to_nlp_df['ingredients'][0],
        signature=signature,        
        artifact_path="sklearn_model",
        artifacts=artifacts
        ) 

    # since this uses a custom Stanza analyzer, we have to use a custom mlflow.Pyfunc.PythonModel
    
    

In [ ]:
response

In [ ]:
test_predictor = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

In [ ]:
# pre_proc_df is cleaned dataframe
pre_proc_test_df = dfpp.preprocess_dataframe(test_df)
print('\n')
print('--------------')
print('Preprocessed Dataframe: ', end='\n')
print(pre_proc_test_df.head())
print(pre_proc_test_df.shape)

# create subset for dev purposes
# to_nlp_test_df = pre_proc_test_df
# print('\n')
# print('-' * 80)
# print('Subset Dataframe:', end='\n')
# print(to_nlp_test_df.head())
# print(to_nlp_test_df.shape)

test_model_input = pre_proc_test_df['ingredients']

In [ ]:
test_model_input

In [ ]:
test_model_input.shape

In [ ]:
test_model_input.values

In [ ]:
model_info.signature.to_dict()

In [ ]:
test_predictor.predict(test_model_input)

In [ ]:
print('\n')
print('-' * 80)
print('Input Data: ', end='\n')
print(test_model_input)

print('\n')
print('-' * 80)
print('Input Data Shape: ', end='\n')
print(test_model_input.shape)

print('\n')
print('-' * 80)
print('Random 3 Records from Input Data: ', end='\n')
print(test_model_input.sample(3, random_state=200))

# test_response = sklearn_transformer.transform(tqdm(test_model_input)) 
test_response = sklearn_transformer.transform(test_model_input)
    
    
test_transformed_recipe = pd.DataFrame(
            test_response.toarray(),
            columns=sklearn_transformer.get_feature_names_out(),
            index=test_model_input.index
    )

In [ ]:
type(test_predictor)

In [ ]:
test_transformed_recipe